In [6]:
!pip install -U langchain langchain-community langchain-huggingface faiss-cpu transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 146.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incomp

In [4]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

In [5]:
# ---------------------------
# 1. Load knowledge base
# ---------------------------
loader = TextLoader("/content/abc.txt")
documents = loader.load()

# ---------------------------
# 2. Split documents into chunks
# ---------------------------
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# ---------------------------
# 3. Create embeddings
# ---------------------------
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ---------------------------
# 4. Store in FAISS and create retriever
# ---------------------------
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

# ---------------------------
# 5. LLM for generating answers
# ---------------------------
qa_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    tokenizer="google/flan-t5-base",
    device=-1  # CPU
)
local_llm = HuggingFacePipeline(pipeline=qa_pipeline)

# ---------------------------
# 6. Conversational memory
# ---------------------------
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ---------------------------
# 7. Create conversational retrieval chain
# ---------------------------
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=local_llm,
    retriever=retriever,
    memory=memory
)

# ---------------------------
# 8. Function to ask the bot
# ---------------------------
def ask_bot(query):
    result = qa_chain({"question": query})
    return result["answer"]


/tmp/ipython-input-3263407303.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

In [6]:
# ---------------------------
# 9. Test
# ---------------------------
print("User: What is LangChain?")
print("Bot:", ask_bot("What is LangChain?"))

print("\nUser: How does FAISS help in LangChain?")
print("Bot:", ask_bot("How does FAISS help in LangChain?"))

User: What is LangChain?


/tmp/ipython-input-3263407303.py:53: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": query})


Bot: LangChain is a framework for building applications with language models.

User: How does FAISS help in LangChain?
Bot: When a question is asked, LangChain retrieves relevant documents from FAISS to answer accurately.


In [4]:
print("User: What i asked previously?")
print("Bot:", ask_bot("What i asked previously?"))

User: What i asked previously?
Bot: LangChain


 ..............

**Using Streamlit**

In [18]:
# Install dependencies
!pip install -q streamlit pyngrok langchain transformers sentence-transformers

import os
import time
from pyngrok import ngrok

# Ngrok auth
ngrok.set_auth_token("32TEOzn2ftdrPTyc9AWh4W8T1lQ_5iKmmMcL12aZfJUtum3Gc")

# Fixed port
port = 8501
os.environ["STREAMLIT_SERVER_PORT"] = str(port)
os.environ["STREAMLIT_SERVER_HEADLESS"] = "true"

# Write Streamlit app
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

st.set_page_config(page_title="Context-Aware Chatbot")
st.title("Context-Aware Chatbot (LangChain + FAISS)")

# ---------------------------
# 1. Load knowledge base
loader = TextLoader("/content/abc.txt")
documents = loader.load()

# 2. Split documents
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# 3. Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. FAISS retriever
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

# 5. LLM
qa_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    tokenizer="google/flan-t5-base",
    device=-1
)
local_llm = HuggingFacePipeline(pipeline=qa_pipeline)

# 6. Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 7. Conversational chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=local_llm,
    retriever=retriever,
    memory=memory
)

# 8. Bot function
def ask_bot(query):
    result = qa_chain({"question": query})
    return result["answer"]

# ---------------------------
# Streamlit UI
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

query = st.text_input("You:", "")

if query:
    answer = ask_bot(query)
    st.session_state.chat_history.append(("User", query))
    st.session_state.chat_history.append(("Bot", answer))

for speaker, text in st.session_state.chat_history:
    if speaker == "User":
        st.markdown(f"**You:** {text}")
    else:
        st.markdown(f"**Bot:** {text}")
""")

# Start Streamlit in background
get_ipython().system_raw(f"streamlit run app.py &")

# Wait a few seconds for Streamlit to fully start
time.sleep(5)

# Start ngrok tunnel
public_url = ngrok.connect(addr=port, proto="http")
print(f"Your working Streamlit URL: {public_url}")


Your working Streamlit URL: NgrokTunnel: "https://91b4f1613a9d.ngrok-free.app" -> "http://localhost:8501"


In [17]:
from pyngrok import ngrok

print(ngrok.get_tunnels())
ngrok.kill()  # stops all ngrok tunnels in this session

[<NgrokTunnel: "https://9800b476c732.ngrok-free.app" -> "http://localhost:8501">, <NgrokTunnel: "https://28348e3b57f9.ngrok-free.app" -> "http://localhost:8501">, <NgrokTunnel: "https://64020da6e9fa.ngrok-free.app" -> "http://localhost:8501">]
